

This coding script makes MRC ANALYSIS
    
----------
# CONTENTS

-    [1. importing necessary libraries](#1)  
             1.1 importing puclic libraries:
                   --if this box is giving an error, this mean you need to check libraries in this cell, 
                   you need    to install them               
             1.2 importing user-defined libraries 
                   --if this box is giving an error, this mean you need to check
                   ipynb file(jupyter notebook) with the name of  'project_helper_functions_classes' and/or
                   MRC_Analysis_classes_functions' within your working directory

-     [2. Preperation of input informations](#2)       
             2.1 Specify folder paths    
                -- This part for setting folder and file names 
                for correctly loading inputs and getting desired output paths
                -- THIS PART COULD BE EDIT depend on your data storage structure.
             2.2 choosing calculation limits              
                 -- This part for choosing limits for MRC calculation
                 -- THIS PART COULD BE EDIT depend on your decision.                          
              2.3 preparing file paths for processing and folders for saving MRC results           
-     [3. MRC analysis and recording results to csv files](#3) 
             3.1 define function for MRC analysis 
             3.2 take MRC results


<a name='1'></a>
## 1. importing necessary libraries

In [3]:
# 1.1 importing puclic libraries
import numpy as np
import pandas as pd
import pathlib
import os

In [4]:
# 1.2 importing user-defined libraries
from ipynb.fs.full.project_helper_functions_classes import take_folder_list, take_special_file_list, create_folder
from ipynb.fs.full.project_helper_functions_classes import create_mirror_subfolders
from ipynb.fs.full.Feature_extraction_classes_functions import Feature_Extraction, check_if_MRC_completed, save_results_to_csv
from ipynb.fs.full.Feature_extraction_classes_functions import run_apply_async_multiprocessing

<a name='2'></a>
## 2. Preperation of input informations 

### 2.1 Specify folder paths

In [7]:
data_root_folder = 'data'
output_root_folder = 'output'      
MRC_output_folder = 'MRC_results'   ## this folder will be created within code as: output_root_folder/MRC_output_folder

### 2.2 choosing calculation limits

In [9]:
min_step, max_step = 1, 5
min_power, max_power = 1, 2
ROI_shapes = ['circle', 'square'] 

## calculation_options for rates
primary_rate = 'all'  # (input options: 'all', 1, 2, 3)
secondary_rate = 'all'  # (input options: 'all', 1, 2, 3, 4, 5, 6, 7, 8)

## calculation limits
calculate_all_powers = True   # (input options: True, False)
calculate_all_steps = True    # (input options: True, False)

### 2.3 preparing file paths for processing and folders for saving MRC results 

In [11]:
stripped_data_folder_path =  os.path.abspath(os.path.join(data_root_folder, 'stripped_data'))
original_data_folder_path =  os.path.abspath(os.path.join(data_root_folder, 'ordered_data'))
mainfolders = take_folder_list(stripped_data_folder_path)

## specify file list for proccess and folder list for record
file_names_list, folder_path_list = take_special_file_list(mainfolders, stripped_data_folder_path, original_data_folder_path)

output_path = os.path.join(os.path.abspath(output_root_folder), MRC_output_folder)
create_folder(output_path)
create_mirror_subfolders(folder_paths =folder_path_list, 
                         old_root_path = stripped_data_folder_path, new_root_path = output_path)
## update files list
file_names_list = [files for files in file_names_list if not check_if_MRC_completed(output_path, files[0], stripped_data_folder_path, min_step, max_step)]


<a name='3'></a>
## 3. MRC analysis and recording to csv files

### 3.1 create functions for MRC analysis by using multiprocessing 

In [14]:
def MRC_analysis(file_names):
    
    ## check if MRC files created before as in desired limits     
    if check_if_MRC_completed(output_path, file_names[0], stripped_data_folder_path, min_step, max_step) :        
        return(0)

    # define inner function to making MRC analysis for different ROI shapes
    def MRC_calculation_(roi_shape):
        fe = Feature_Extraction(stripped = file_names[0], original = file_names[1], 
                                min_power = min_power, max_power = max_power,
                                min_step = min_step, max_step = max_step,
                                ROI_shape = roi_shape, 
                                primary_rate = primary_rate, secondary_rate = secondary_rate,
                                all_powers = calculate_all_powers, all_steps = calculate_all_steps)
        fe.calculate_features(calculate_MRC = True)
        return(fe.MRC_results)

    results = list(map(MRC_calculation_, ROI_shapes))

     # unite different ROI shape results if 2 shape are givev
    if len(results) == 2:
        MRC_results = pd.concat([results[0],  results[1]], ignore_index=True) # unite different ROI results

    if len(results) == 1:
        MRC_results = results[0]


    save_results_to_csv(MRC_results, output_path, file_names[0], stripped_data_folder_path)



### 3.2 take MRC results

In [16]:
if __name__ == '__main__':
    
    run_apply_async_multiprocessing(MRC_analysis, file_names_list, 1)

100%|█████████████████████████████████████████| 303/303 [22:56<00:00,  4.54s/it]
